In [3]:
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split, cross_val_score, KFold
import pandas as pd


df = pd.read_csv(r"D:\dtc-dr\data-analyse\continuous_factory_process.csv", delimiter=",")
prefixes_to_match = ["Machine1", "Machine2", "Machine3"]

filtered_columns = [
    col
    for col in df.columns
    if any(col.startswith(prefix) for prefix in prefixes_to_match)
]

# Assuming X_train and y_train are your training data

X = df[filtered_columns]

best_indexes = [0, 3, 7, 8, 24, 25, 26, 27, 32]

X = [X.columns.tolist()[index] for index in best_indexes]
X = df[X]

X.columns.tolist()

Y = df['Stage1.Output.Measurement1.U.Actual']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
# Define the parameter grid to search
param_grid = {
    'hidden_layer_sizes': [(100,), (50, 50), (100, 50, 25)],  # Example values, adjust as needed
    'activation': ['relu', 'tanh'],
    'alpha': [0.0001, 0.001, 0.01],
    'learning_rate': ['constant', 'invscaling', 'adaptive']
}

# Create the MLP regressor
mlp_regr = MLPRegressor(max_iter=1000)  # You can adjust max_iter based on your needs

# Create the GridSearchCV object
grid_search = GridSearchCV(mlp_regr, param_grid, cv=5, scoring='r2', n_jobs=-1)

# Fit the model to the data
grid_search.fit(X_train, y_train)

# Get the best parameters and the best estimator
best_params = grid_search.best_params_
best_mlp_regr = grid_search.best_estimator_

# Print the best parameters
print("Best hyperparameters for MLP:", best_params)

# Now you can use the best estimator for further analysis
# Example:
r2_scores_cv = cross_val_score(best_mlp_regr, X_train, y_train, cv=5, scoring='r2')
print("Cross-validated R-squared scores for MLP with tuned hyperparameters:", r2_scores_cv)


Best hyperparameters for MLP: {'activation': 'tanh', 'alpha': 0.0001, 'hidden_layer_sizes': (100,), 'learning_rate': 'adaptive'}
Cross-validated R-squared scores for MLP with tuned hyperparameters: [0.75779317 0.72172634 0.62415145 0.67580437 0.76259452]
